In [ ]:
# default_exp core

In [ ]:
import nbdev.showdoc as literacy

In [ ]:
#export
from speechsep.imports import *
import speechsep.utils as utils
import speechsep.plot as plot

# Core

This contains most of the basic functions and spectrogram class types. To visualize the spectrograms we will also include a special color map since this makes it easier to notice differences in audio intensities.

The most important things to remember are
- How to create an AudioItem both from a numpy array and from file.
- Creating a SpecImage and how the parameters influence the final result.
- Basic SpecImage Visualizer (more indepth explanation here***)

## Loading Data

In [ ]:
#export
@delegates(load)
def load_audio(fn, **kwargs):
    return load(fn)

In [ ]:
fn = Path("../data/AudioTest1.wav")
sig, sr = load_audio(fn)
display(Audio(sig, rate=sr))

test_eq(type(sig), np.ndarray)
test_eq(type(sr), int)

## AudioItem

In [ ]:
class AudioBase():
    def __init__(self,sig,_sr,fn=None):
        store_attr(self, 'sig,_sr,fn')
    def __repr__(self): display(Audio(self.sig, rate=self.sr)); return f'{self.__str__()}'
    def __str__(self): return f'{self.fn}, {len(self.sig)/self.sr}secs at {self.sr} samples per second'
    @delegates(Line2D)
    def show(self, **kwargs): plt.plot(self.sig, **kwargs)

In [ ]:
#export
class AudioItem(AudioBase):
    @classmethod
    def create(cls, fn, sr=None):
        audio = cls(*load_audio(fn),fn)
        if sr: audio.sr = sr
        return audio
    load_file = create
    @property
    def sr(self): return self._sr
    @sr.setter
    def sr(self, new_sr):
        if self._sr != new_sr: self.sig = utils.Resample(new_sr)(self.sig, self.sr)
        self._sr = new_sr

In [ ]:
aud1 = AudioItem.create(fn)

test_eq(type(aud1), AudioItem)
test_eq(aud1.sr, 22050)
test_eq(aud1.fn, fn)

In [ ]:
aud2 = AudioItem.create(fn, sr=2205)

test_eq(aud2.sr, 2205)
test_eq(type(aud2.sig), np.ndarray)
test_eq(type(aud2.sr), int)

In [ ]:
@patch_property
def duration(x:AudioItem):
    return len(x.sig)/x.sr

In [ ]:
test_eq(type(aud1.duration), float)
test_eq(round(aud1.duration), 4)

In [ ]:
aud1.sr = 48000

test_eq(aud1.sr, 48000)
test_eq(round(aud1.duration), 4)

## Spectrograms

In [ ]:
#export
class SpecBase():
    def __init__(self, data, sr, fn=None):
        store_attr(self, 'data, sr, fn')
        self._plt_params = {}
    @property
    def plt_params(self): return self._plt_params
    @plt_params.setter
    @delegates(plt.pcolormesh)
    def plt_params(self, **kwargs):
        self._plot = partial(plt.pcolormesh, **kwargs)
        self._plt_params = dict(**kwargs)

In [ ]:
#export
class Spectify(core.Transform):
    def __init__(self, fftsize=512, win_mult=2, overlap=0.5, freq='linear', amp='linear'):
        store_attr(self, 'fftsize, win_mult, overlap, freq, amp')
    def encodes(self, audio:AudioItem):
        spec = utils.stft(audio.sig, self.fftsize, self.win_mult, self.overlap)
        return SpecBase(spec, audio.sr, audio.fn)
    def decodes(self, spec):
        audio = utils.istft(spec.data, self.fftsize, self.win_mult, self.overlap)
        return AudioItem(audio, spec.sr, spec.fn)

    @patch
    @classmethod
    @delegates(to=Spectify)
    def create(x:SpecBase, fn, sr=None, name=None, **kwargs):
        #Open an `Audio` from path `fn`
        if isinstance(fn,(Path,str)): return cls.create(AudioItem(fn, sr), name=name)
        elif isinstance(fn,AudioItem): return Spectify(**kwargs)(fn)
        elif isinstance(fn,np.ndarray): return cls(fn, sr, name)
        raise ValueError('fn must be ndarray, AudioItem or Path')

In [ ]:
audio = AudioItem.load_file(fn)
Audio2Spec = Spectify()
spec = Audio2Spec(audio)

In [ ]:
test_eq(type(spec), SpecBase)
test_eq(type(spec.data), np.ndarray)
test_eq(spec.fn, fn)
test_eq(spec.sr, 22050)

In [ ]:
#export
@patch
@delegates(plot.setup_graph)
def show(x:SpecBase, ctx=None, **kwargs):
    plot.setup_graph(**kwargs)
    plt.pcolormesh(abs(x.data[:x.data.shape[0]//2]))

In [ ]:
spec.show(fig_size = [12,8])

In [ ]:
audio_r = Audio2Spec.decodes(spec)
audio_r

In [ ]:
test_eq(type(audio_r), AudioItem)
test_eq(type(audio_r.sig), np.ndarray)
test_eq(audio_r.sr, 22050)
test_eq(audio_r.fn, fn)